# Prototype
#### Import necessary packages:

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import math

import xgboost as xgb
from sklearn import linear_model
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error

In /Users/cathleenpena1/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/cathleenpena1/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/cathleenpena1/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/cathleenpena1/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/cathleenpena1/anac

### Load Data

In [3]:
df = pd.read_csv('data/argentina_final.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head(3)

,start_date,end_date,created_on,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,...,property_type_House,property_type_Lot,property_type_Office,property_type_Other,property_type_PH,property_type_Shop,property_type_Warehouse,operation_type_For Rent,operation_type_For Sale,operation_type_For Sublease
0,2020-06-17,2020-06-18,2020-06-17,-34.995340,-58.047423,2.844661,0.0,1.70148,474.884308,6370.15545,...,0,0,0,0,0,0,0,0,1,0
1,2020-05-20,2020-05-20,2020-05-20,-34.816634,-59.193616,5.000000,2.0,1.00000,72.000000,72.00000,...,0,0,0,0,0,0,0,0,1,0
2,2020-07-14,2020-08-13,2020-07-14,-31.116769,-64.482921,2.844661,3.0,1.00000,747.000000,244.00000,...,1,0,0,0,0,0,0,0,1,0


### Split data

In [11]:
train_percent = .7
valid_test_percent = .15
n_rows = len(df)

train_stop = round(n_rows * train_percent)
valid_stop = round(n_rows * train_percent + n_rows * valid_test_percent)

Xtrain = df[:train_stop].drop('price', axis=1)
ytrain = df[:train_stop]['price']

Xvalid = df[train_stop:valid_stop].drop('price', axis=1)
yvalid = df[train_stop:valid_stop]['price']

Xtest = df[valid_stop:].drop('price', axis=1)
ytest = df[valid_stop:]['price']

print("Train Percent", len(Xtrain) / n_rows)
print("Valid Percent", len(Xvalid) / n_rows)
print("Test Percent", len(Xtest) / n_rows)

Train Percent 0.7000004153263738
Valid Percent 0.14999927317884576
Test Percent 0.15000031149478038


## Create Model

## Evaluate Model 
Use correct metrics

cross val score (k-fold cross validation)

## Hyperparameter Tuning
Grid Search CV

# Compare to base model and untuned model
Metrics and Visualizations